In [ ]:

import pandas as pd
import numpy as np
import re


initial_data = pd.read_csv("Snappfood.csv", sep="\t", encoding="utf-8-sig")
filtered_data = initial_data.dropna(subset=["comment", "label", "label_id"])
new_data = pd.DataFrame()
new_data["comment"] = filtered_data["comment"].dropna().values
new_data["label"] = filtered_data["label"].str.lower().dropna().values
new_data["label_id"] = filtered_data["label_id"].dropna().values.astype(np.int8)

new_data.to_csv("new_data.csv",encoding="utf-8-sig" ,index=False)

def is_persian(word):
    return bool(re.fullmatch(r'[آ-یءئاآبپتثجچحخدذرزژسشصضطظعغفقکگلمنوهی]+', word))


temp_data = pd.read_csv("new_data.csv", encoding="utf-8-sig")
sad_data = temp_data[(temp_data["label_id"] == 1)]

happy_data = temp_data[(temp_data["label_id"] == 0)]
sad_temp = sad_data["comment"].str.split()

happy_temp = happy_data["comment"].str.split()
happy_flat_words = np.hstack(happy_temp.dropna().tolist())
sad_flat_words = np.hstack(sad_temp.dropna().tolist())

happy_persian = [w for w in happy_flat_words if is_persian(w)]
sad_persian = [w for w in sad_flat_words if is_persian(w)]

unique_words, counts = np.unique(happy_persian, return_counts=True)
happy_df = pd.DataFrame({
    "word": unique_words,
    "count_happy": counts
})
unique_words, counts = np.unique(sad_persian, return_counts=True)
sad_df = pd.DataFrame({
    "word": unique_words,
    "count_sad": counts
})
happy_df.to_csv("happy_words.csv", encoding="utf-8-sig", index=False)
sad_df.to_csv("sad_words.csv", encoding="utf-8-sig", index=False)

happy_words = pd.read_csv("happy_words.csv")
sad_words = pd.read_csv("sad_words.csv")

result_df = pd.merge(happy_words, sad_words, on="word", how="outer")
result_df = result_df.fillna(0)

total_happy_words = happy_words["count_happy"].sum()
total_sad_words = sad_words["count_sad"].sum()

result_df["percent_happy"] = ((result_df["count_happy"] / total_happy_words) * 100).round(3)
result_df["percent_sad"] = ((result_df["count_sad"] / total_sad_words) * 100).round(3)
result_df["diff"] = (abs(result_df["percent_happy"] - result_df["percent_sad"])).round(3)
